In [1]:
#NLP
import spacy
nlp = spacy.load("es_core_news_sm")
print(spacy.__version__)
from spacy.lang.es.stop_words import STOP_WORDS
from spacy.lang.es import Spanish
import string

#SKLEARN
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.linear_model import LogisticRegression # Regresion Logística

#PANDAS
import pandas as pd

3.1.3


In [31]:
df = pd.read_csv('datos/sophia2_data_23112021.csv')

In [52]:
df

,Unnamed: 0,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Dominant_Topic_Name,Dominant_Topic_REF
0,0,0,6.0,0.9859,"partido, equipo, jugar, jugador, club, chileno...",queda cerca de un mes para que comience el tor...,Deporte,7
1,1,1,14.0,0.9700,"región, caso, comuna, salud, sanitario, cuaren...",el gobierno anunció cambios en comunas de todo...,Salud,10
2,2,2,5.0,0.4317,"millón, empresa, económico, mes, mercado, dóla...","con un promedio de un 25% de desempleo, latino...",Economia,2
3,3,3,10.0,0.3685,"agua, zona, proyecto, vehículo, ambiental, exp...",¿los aficionados al queso pueden seguir disfru...,Ecologia y Planeta,8
4,4,4,15.0,0.9380,"vacuna, salud, dosis, vacunación, médico, estu...",el ministerio de salud aseguró este jueves que...,Salud,10
...,...,...,...,...,...,...,...,...
50465,50465,50465,1.0,0.9439,"carabinero, encontrar, vehículo, policía, homb...","durante la tarde de este martes, efectivos de ...","Crimen, Delitos y Justicia",9
50466,50466,50466,1.0,0.5740,"carabinero, encontrar, vehículo, policía, homb...",el primer tribunal de juicio oral en lo penal ...,"Crimen, Delitos y Justicia",9
50467,50467,50467,14.0,0.9956,"región, caso, comuna, salud, sanitario, cuaren...",este viernes las autoridades de salud entregar...,Salud,10
50468,50468,50468,15.0,0.3845,"vacuna, salud, dosis, vacunación, médico, estu...","el ministro de salud, enrique paris, reproch...",Salud,10


In [50]:
from pandasql import sqldf

In [51]:
query_sql="SELECT Dominant_Topic_Name, count(*) FROM df GROUP BY Dominant_Topic_Name ORDER BY count(*) DESC;"
df_result=sqldf(query_sql)
df_result

,Dominant_Topic_Name,count(*)
0,Salud,10449
1,"Crimen, Delitos y Justicia",9354
2,Politica y Conflictos,8861
3,Cultura y Artes,6567
4,Deporte,5078
5,Mundo,3287
6,Economia,2755
7,Ecologia y Planeta,2640
8,Catastrofes y Accidentes,1479


In [33]:
X = df['Text'] # the features we want to analyze
ylabels = df['Dominant_Topic_Name'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.5)

In [34]:
# Create our list of punctuation marks
punctuations = [".",",","!","?", "#","&"]

# Create our list of stopwords
stop_words=[""]

# Load English tokenizer, tagger, parser, NER and word vectors
parser = Spanish()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lower_ for word in mytokens]
        
    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [35]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
bow_vector

CountVectorizer(tokenizer=<function spacy_tokenizer at 0x000002B02EBBBA60>)

In [39]:
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(random_state=0)

model3 = Pipeline([('preprocessing', bow_vector),
                 ('regression-ML', modelRF)])

# model generation
model3.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x000002B02EBBBA60>)),
                ('regression-ML', RandomForestClassifier(random_state=0))])

In [42]:
# Predicting with a test dataset
predicted = model3.predict(X_test)
print(predicted)

# Model Accuracy
print("Logistic Regression Accuracy:", metrics.accuracy_score(y_test, predicted))
#print("Logistic Regression Precision:", metrics.precision_score(y_test, predicted))
#print("Logistic Regression Recall:", metrics.recall_score(y_test, predicted))

['Salud' 'Salud' 'Deporte' ... 'Deporte' 'Crimen, Delitos y Justicia'
 'Salud']
Logistic Regression Accuracy: 0.8145829205468595


In [43]:
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, predicted)
print(confusion_matrix)

#Print de la matriz de confusión
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted))

[[ 265  286   33    5    2    1    1    7  161]
 [   1 4238   17   10    2    0   14  233  151]
 [   1  184 2668  138   11    2   20   82  139]
 [   0    7   24 2549    0    1    0    4   19]
 [   6  197  121   10  480   11   11   67  359]
 [   0  147   52   16    9  466   35  218  439]
 [   0  250   43    7    1    1  960  186  151]
 [   0  133   18   18    0    2   82 4009  219]
 [   1  115   34   12    1    3   14  134 4921]]
                            precision    recall  f1-score   support

  Catastrofes y Accidentes       0.97      0.35      0.51       761
Crimen, Delitos y Justicia       0.76      0.91      0.83      4666
           Cultura y Artes       0.89      0.82      0.85      3245
                   Deporte       0.92      0.98      0.95      2604
        Ecologia y Planeta       0.95      0.38      0.54      1262
                  Economia       0.96      0.34      0.50      1382
                     Mundo       0.84      0.60      0.70      1599
     Politica y Confli